# Download do arquivo

In [5]:
# Entrada no shell
# ! precisa do sdk google cloud instalado
# !baixa o arquivo do bucket para a pasta atual
!gsutil -m cp "gs://gcs_dataeng_ecomm_mono_arqs/product_teste_rmg.json" ./

Copying gs://gcs_dataeng_ecomm_mono_arqs/product_teste_rmg.json...
/ [0/1 files][    0.0 B/  8.1 KiB]   0% Done                                    
/ [1/1 files][  8.1 KiB/  8.1 KiB] 100% Done                                    

Operation completed over 1 objects/8.1 KiB.                                      


# Imports

In [18]:
import json
import io
from google.cloud import bigquery
from bson import json_util as bj_util

# Cria tabela tm

In [8]:
# cria tabela 
!bq cp -f ssbx-dataeng-ecomm-multi:raw_amazon_blz.product_streaming_json ssbx-dataeng-ecomm-multi:raw_amazon_blz.rmg_streaming_json_tt

Table 'ssbx-dataeng-ecomm-multi:raw_amazon_blz.product_streaming_json' successfully copied to 'ssbx-dataeng-ecomm-multi:raw_amazon_blz.rmg_streaming_json_tt'



Waiting on bqjob_r66e03344ce199f14_00000192bfb01e3f_1 ... (0s) Current status: RUNNING
                                                                                      
Waiting on bqjob_r66e03344ce199f14_00000192bfb01e3f_1 ... (0s) Current status: DONE   


# Dump sensitive-raw.raw_amazon_blz.product_streaming

In [37]:
load_vars = json.dumps({
    "bq":{
        "project_id": "ssbx-dataeng-ecomm-multi",
        "dataset_id": "raw_amazon_blz",
        "target_table": "rmg_streaming_json_tt",
        "destination_table_description": "Essa tabela contem informações dos pacotes de transporte disponibilizadas no ecommerce"
    },
    "source":{
        "file": "./product_teste_rmg.json"
    }
})
print(load_vars)

{"bq": {"project_id": "ssbx-dataeng-ecomm-multi", "dataset_id": "raw_amazon_blz", "target_table": "rmg_streaming_json_tt", "destination_table_description": "Essa tabela contem informa\u00e7\u00f5es dos pacotes de transporte disponibilizadas no ecommerce"}, "source": {"file": "./product_teste_rmg.json"}}


In [20]:
schema=[
    bigquery.SchemaField(
        name='subscription_name',
        field_type='STRING',
        mode='NULLABLE',
        description=None
    ),
    bigquery.SchemaField(
        name='message_id',
        field_type='STRING',
        mode='NULLABLE',
        description=None
    ),
    bigquery.SchemaField(
        name='publish_time',
        field_type='TIMESTAMP',
        mode='NULLABLE',
        description=None
    ),
    bigquery.SchemaField(
        name='data',
        field_type='STRING',
        mode='NULLABLE',
        description=None
    ),
    bigquery.SchemaField(
        name='attributes',
        field_type='STRING',
        mode='NULLABLE',
        description=None
    )
]

In [11]:
def json_new_line_delimited_to_bytes(data) -> io:
    """Helper for JSON to Bytes converter after request, used in extract rewrites
    """
    new_line_delimited_json = '\n'.join([json.dumps(item) for item in data])
    return io.BytesIO(new_line_delimited_json.encode("utf-8"))

In [21]:
def fake_raw_event(data):
    raw_model = []
    publish_time = "2024-10-01 04:05:00"
    for row in data:
        item ={
            "subscription_name":"Ingestao_manual",
            "message_id":"5000000000000000",
            "publish_time":publish_time,
            "data":row,
            "attributes":'{"descricao":"Dump dos eventos de produtos da plataforma BLZ refente ao periodo de falha no CDC no inicio de Outubro 2024"}'
        }
        raw_model.append(item)
    return json_new_line_delimited_to_bytes(raw_model)

In [22]:
params = json.loads(load_vars)
table_id = f'{params["bq"]["project_id"]}.{params["bq"]["dataset_id"]}.{params["bq"]["target_table"]}'
print (table_id)

ssbx-dataeng-ecomm-multi.raw_amazon_blz.rmg_streaming_json_tt


In [23]:
# Construct a BigQuery client object.
client = bigquery.Client(project=params["bq"]["project_id"])

In [59]:
# Configuração do job de escrita no BigQuery
job_config = bigquery.LoadJobConfig(
    schema=schema,
    write_disposition=bigquery.WriteDisposition.WRITE_APPEND,
    source_format=bigquery.SourceFormat.NEWLINE_DELIMITED_JSON
)
data = []
# Lendo arquivo json
with open(params['source']['file'], 'r') as file:
    data = json.loads(bj_util.dumps(file))

print(data) 

['{ "_id" : { "$oid" : "111f30" }, "sku" : "111", "createdAt" : { "$date" : "2020-01-20T15:37:33.348+0000" } }\n', '{ "_id" : { "$oid" : "222f51" }, "sku" : "222", "createdAt" : { "$date" : "2020-01-20T15:37:33.348+0000" } }\n', '{ "_id" : { "$oid" : "333f5a" }, "sku" : "333", "createdAt" : { "$date" : "2020-01-20T15:37:33.348+0000" } }']


In [58]:
#file_obj=fake_raw_event(data)
#print(file_obj)
print(table_id)
destination_table = client.get_table(table_id)
print("Loaded {} rows.".format(destination_table.num_rows))

ssbx-dataeng-ecomm-multi.raw_amazon_blz.rmg_streaming_json_tt


RefreshError: ('invalid_grant: Bad Request', {'error': 'invalid_grant', 'error_description': 'Bad Request'})

In [51]:
# Carregando dados no BQ
# file_obj é uma lista em bytes delimitados por new line com  o resultado
# do input do arquivo json sem virgula na função fake_raw_event
load_job = client.load_table_from_file(
    file_obj=fake_raw_event(data),
    destination=table_id,
    location="US",  # Must match the destination dataset location.
    job_config=job_config,
)  # Make an API request.

load_job.result()  # Waits for the job to complete.

destination_table = client.get_table(table_id)
print("Loaded {} rows.".format(destination_table.num_rows))

RefreshError: ('invalid_grant: Bad Request', {'error': 'invalid_grant', 'error_description': 'Bad Request'})

In [52]:
# Em caso de erro esse bloco permite depurar os retornos
load_job.errors

NameError: name 'load_job' is not defined